In [163]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np

In [164]:
data = pd.read_csv('etfs_dataset_modelling.csv')

In [165]:
data.dropna(inplace=True)
data = data.rename(columns={'Unnamed: 0': 'Series'})
data

,Series,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI_9,RSI_14,RSI_25,ewma_7,ewma_50,ewma_200,Close_target
25,25,FGDL,2022-08-05 00:00:00-04:00,23.820000,23.820000,23.768000,23.768000,210200.0,0.0,0.0,96.519827,57.326070,0.000000,23.655462,23.651515,23.995224,23.952000
26,26,FGDL,2022-08-08 00:00:00-04:00,23.900999,23.952000,23.900999,23.952000,200.0,0.0,0.0,100.000000,62.375775,0.000000,23.729596,23.663299,23.994794,24.049999
27,27,FGDL,2022-08-09 00:00:00-04:00,24.016001,24.070000,24.016001,24.049999,11300.0,0.0,0.0,100.000000,67.369977,38.395312,23.809697,23.678464,23.995344,23.975000
28,28,FGDL,2022-08-10 00:00:00-04:00,23.972000,23.975000,23.972000,23.975000,200.0,0.0,0.0,100.000000,78.593118,62.521736,23.851023,23.690093,23.995141,23.913000
29,29,FGDL,2022-08-11 00:00:00-04:00,23.913000,23.913000,23.913000,23.913000,0.0,0.0,0.0,100.000000,89.394880,72.365015,23.866517,23.698834,23.994324,24.110001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3571480,3571480,AUGT,2024-07-02 00:00:00-04:00,29.120001,29.195000,29.120001,29.195000,200.0,0.0,0.0,100.000000,100.000000,100.000000,29.097374,28.455972,26.980250,29.267000
3571481,3571481,AUGT,2024-07-03 00:00:00-04:00,29.221001,29.270000,29.180000,29.267000,2300.0,0.0,0.0,100.000000,100.000000,100.000000,29.139780,28.487777,27.003004,29.297001
3571482,3571482,AUGT,2024-07-05 00:00:00-04:00,29.240000,29.297001,29.240000,29.297001,500.0,0.0,0.0,100.000000,100.000000,100.000000,29.179086,28.519512,27.025830,29.332001
3571483,3571483,AUGT,2024-07-08 00:00:00-04:00,29.299999,29.340000,29.299999,29.332001,600.0,0.0,0.0,100.000000,100.000000,100.000000,29.217314,28.551374,27.048777,29.327000


##### Extraer fechas en formato que lean las series y eliminamos columnas no necesarias

In [166]:
data['Date'] = pd.to_datetime(data['Date'], utc = True)

In [167]:
data['Date'] = data['Date'].dt.date
data['Day'] = [i.day for i in data['Date']]
data['Month'] = [i.month for i in data['Date']]
data['Year'] = [i.year for i in data['Date']]

data = data[['Series', 'Year', 'Month', 'Day', 'Close', 'Ticker']] 

data.head()

,Series,Year,Month,Day,Close,Ticker
25,25,2022,8,5,23.768000,FGDL
26,26,2022,8,8,23.952000,FGDL
27,27,2022,8,9,24.049999,FGDL
28,28,2022,8,10,23.975000,FGDL
29,29,2022,8,11,23.913000,FGDL


##### Predicción Naive

Primero dividimos en train y test de forma manual

In [168]:
prueba = [data for data in data.groupby('Ticker')]
prueba

[('AAA',
            Series  Year  Month  Day      Close Ticker
  1173759  1173759  2020     10   14  24.703897    AAA
  1173760  1173760  2020     10   15  24.703897    AAA
  1173761  1173761  2020     10   16  24.703897    AAA
  1173762  1173762  2020     10   19  24.698940    AAA
  1173763  1173763  2020     10   20  24.703897    AAA
  ...          ...   ...    ...  ...        ...    ...
  1174692  1174692  2024      7    2  25.049999    AAA
  1174693  1174693  2024      7    3  25.049999    AAA
  1174694  1174694  2024      7    5  25.056000    AAA
  1174695  1174695  2024      7    8  25.059999    AAA
  1174696  1174696  2024      7    9  25.030001    AAA
  
  [938 rows x 6 columns]),
 ('AAAU',
            Series  Year  Month  Day      Close Ticker
  2826054  2826054  2018      9   20  12.060000   AAAU
  2826055  2826055  2018      9   21  11.990000   AAAU
  2826056  2826056  2018      9   24  12.000000   AAAU
  2826057  2826057  2018      9   25  12.010000   AAAU
  2826058  28260

In [169]:
prueba[0][1].drop('Ticker', inplace=True, axis=1)

In [170]:
prueba

[('AAA',
            Series  Year  Month  Day      Close
  1173759  1173759  2020     10   14  24.703897
  1173760  1173760  2020     10   15  24.703897
  1173761  1173761  2020     10   16  24.703897
  1173762  1173762  2020     10   19  24.698940
  1173763  1173763  2020     10   20  24.703897
  ...          ...   ...    ...  ...        ...
  1174692  1174692  2024      7    2  25.049999
  1174693  1174693  2024      7    3  25.049999
  1174694  1174694  2024      7    5  25.056000
  1174695  1174695  2024      7    8  25.059999
  1174696  1174696  2024      7    9  25.030001
  
  [938 rows x 5 columns]),
 ('AAAU',
            Series  Year  Month  Day      Close Ticker
  2826054  2826054  2018      9   20  12.060000   AAAU
  2826055  2826055  2018      9   21  11.990000   AAAU
  2826056  2826056  2018      9   24  12.000000   AAAU
  2826057  2826057  2018      9   25  12.010000   AAAU
  2826058  2826058  2018      9   26  11.950000   AAAU
  ...          ...   ...    ...  ...        .

In [175]:
prueba[0][1]

,Series,Year,Month,Day,Close
1173759,1173759,2020,10,14,24.703897
1173760,1173760,2020,10,15,24.703897
1173761,1173761,2020,10,16,24.703897
1173762,1173762,2020,10,19,24.698940
1173763,1173763,2020,10,20,24.703897
...,...,...,...,...,...
1174692,1174692,2024,7,2,25.049999
1174693,1174693,2024,7,3,25.049999
1174694,1174694,2024,7,5,25.056000
1174695,1174695,2024,7,8,25.059999


In [179]:
prueba_1etf = np.array(prueba[0][1])
prueba_1etf, len(prueba_1etf)

(array([[1.17375900e+06, 2.02000000e+03, 1.00000000e+01, 1.40000000e+01,
         2.47038975e+01],
        [1.17376000e+06, 2.02000000e+03, 1.00000000e+01, 1.50000000e+01,
         2.47038975e+01],
        [1.17376100e+06, 2.02000000e+03, 1.00000000e+01, 1.60000000e+01,
         2.47038975e+01],
        ...,
        [1.17469400e+06, 2.02400000e+03, 7.00000000e+00, 5.00000000e+00,
         2.50559998e+01],
        [1.17469500e+06, 2.02400000e+03, 7.00000000e+00, 8.00000000e+00,
         2.50599995e+01],
        [1.17469600e+06, 2.02400000e+03, 7.00000000e+00, 9.00000000e+00,
         2.50300007e+01]]),
 938)

##### Vamos a coger un ticker y hacer el Naive

In [ ]:
def data_train_test(series):

    series = series
    n_steps = 50
    pct_train = np.int_(len(series)*0.7)
    pct_test = np.int_(len(series)*0.2)
    pct_val = np.int_(len(series)*0.1)


    X_train, y_train = series[:pct_train, :n_steps], series[:pct_train, 4]
    last_row = len(X_train)+pct_val

    X_val, y_val = series[len(X_train):last_row, :n_steps], series[len(X_train):last_row, 4]

    X_test, y_test = series[last_row: , :n_steps], series[last_row: , 4]

    return X_train, X_test, y_train, y_test, X_val, y_val


# La suma de las 3 longitudes debe dar 938
X_train = data_train_test(prueba_1etf)[0]
X_test = data_train_test(prueba_1etf)[1]
y_train = data_train_test(prueba_1etf)[2]
y_test = data_train_test(prueba_1etf)[3]
X_val = data_train_test(prueba_1etf)[4]
y_val = data_train_test(prueba_1etf)[5]

def plot_series(series, y=None, y_pred=None, y_pred_std=None, x_label="$t$", y_label="$x$"):
  r, c = 3, 5
  fig, axes = plt.subplots(nrows=r, ncols=c, sharey=True, sharex=True, figsize=(20, 10))
  for row in range(r):
    for col in range(c):
        plt.sca(axes[row][col])
        ix = col + row*c
        plt.plot(series[ix, :], ".-")
        if y is not None:
            plt.plot(range(len(series[ix, :]), len(series[ix, :])+len(y[ix])), y[ix], "bx", markersize=10)
        if y_pred is not None:
            plt.plot(range(len(series[ix, :]), len(series[ix, :])+len(y_pred[ix])), y_pred[ix], "ro")
        if y_pred_std is not None:
            plt.plot(range(len(series[ix, :]), len(series[ix, :])+len(y_pred[ix])), y_pred[ix] + y_pred_std[ix])
            plt.plot(range(len(series[ix, :]), len(series[ix, :])+len(y_pred[ix])), y_pred[ix] - y_pred_std[ix])
        plt.grid(True)
        plt.hlines(0, 0, 100, linewidth=1)
        plt.axis([0, len(series[ix, :])+len(y[ix]), -1, 1])
        if x_label and row == r - 1:
          plt.xlabel(x_label, fontsize=16)
        if y_label and col == 0:
          plt.ylabel(y_label, fontsize=16, rotation=0)
  plt.show()

plot_series(X_test, y_test)
print(len(X_train), len(X_test), len(X_val))
print(len(y_train), len(y_test), len(y_val))
len(X_train)+len(X_test)+len(X_val)
X_test[:,0]
y_test
from sklearn.metrics import mean_squared_error

y_pred = X_test[:,-1]
mean_squared_error(y_test, y_pred)

In [184]:
from sklearn.metrics import mean_squared_error

y_pred = X_test[:,-1]
mean_squared_error(y_test, y_pred)

ValueError: could not convert string to float: 'AAA'

##### Redes Neuronales 

Como en Naive, debemos seleccionar el numero de entradas/registros (fragmentar cada serie). Como cada ETF tiene una longitud de series distinta, cada modelo deberá tener un n_in distinto y adaptado al len(dataset)

##### Redes Neuronales Recurrentes

Indicamos el tamaño de la entrada
.cuda() --> para usar GPU (collab tiene gratis)
En PyTorch, nuestro tensor puede definirse con la informacion de GPU

#####
Train y Test

In [ ]:
X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)